In [ ]:
import ee
import geemap


In [1]:
# print the files in the /projectnb/modislc/users/chishan/data
!ls /projectnb/modislc/users/chishan/data/MapBiomas/


Final  MAPBIOMAS


In [2]:
mapbiomas_dir = '/projectnb/modislc/users/chishan/data/MapBiomas/Final'
#read the csv file
import pandas as pd
mapbiomas_df = pd.read_csv(f'{mapbiomas_dir}/Mapbiomass_all_2022_09_27.csv')
mapbiomas_df.head()


,End_Year,Start_Year,Glance_Class_ID_level1,Glance_Class_ID_level2,Lon,Lat,Level1_Ecoregion,Continent,Continent_Code,Dataset,Dataset_Code,Level2_Ecoregion,Glance_ID,LC_Class,ID
0,2020,1985,5,NaN,-65.408087,-10.000809,66,SA,2,MapBiomas,5,66_00,02_66_00_5_0000000_1985_2020,Forest,0
1,2020,1985,5,NaN,-66.488220,-10.002156,66,SA,2,MapBiomas,5,66_00,02_66_00_5_0000001_1985_2020,Forest,1
2,2020,1985,5,NaN,-65.468455,-10.004582,66,SA,2,MapBiomas,5,66_00,02_66_00_5_0000002_1985_2020,Forest,2
3,2020,1985,5,NaN,-66.470972,-10.004582,66,SA,2,MapBiomas,5,66_00,02_66_00_5_0000003_1985_2020,Forest,3
4,2020,1985,5,NaN,-66.740467,-10.004854,66,SA,2,MapBiomas,5,66_00,02_66_00_5_0000004_1985_2020,Forest,4


In [3]:
mapbiomas_dir = '/projectnb/modislc/users/chishan/data/MapBiomas/MAPBIOMAS'
#read the csv file
import pandas as pd
mapbiomas_df = pd.read_csv(f'{mapbiomas_dir}/samples-BRAZIL.csv')
mapbiomas_df.head()


,reference,year,Lat,Lon
0,4,1985,-11.065127,-46.399067
1,3,1985,-30.637130,-52.121818
2,3,1985,-30.887491,-52.863467
3,12,1985,-30.910937,-52.694494
4,3,1985,-31.017657,-53.559841


In [4]:
mapbiomas_df[mapbiomas_df['year']>2000]


,reference,year,Lat,Lon
1362432,19,2001,-11.065127,-46.399067
1362433,3,2001,-30.637130,-52.121818
1362434,3,2001,-30.887491,-52.863467
1362435,12,2001,-30.910937,-52.694494
1362436,3,2001,-31.017657,-53.559841
...,...,...,...,...
2895163,3,2018,-22.524535,-42.970051
2895164,3,2018,-22.928777,-42.616744
2895165,15,2018,-22.355292,-43.692027
2895166,33,2018,-22.901019,-42.109825


In [5]:
X=mapbiomas_df['reference'].unique()
X.sort()
X


array([ 3,  4,  5,  9, 11, 12, 13, 15, 19, 20, 23, 24, 25, 27, 29, 30, 31,
       32, 33])

In [ ]:
!ps -u $USER -o pid,ppid,%cpu,%mem,cmd --sort=-%mem


In [ ]:
ln -s /projectnb/modislc/users/chishan/data project_data


In [6]:
# Cell: filter rows where reference == 15 (robust to column name)
import pandas as pd
# ensure mapbiomas_df exists in the notebook namespace
if 'mapbiomas_df' not in globals():
    raise NameError('mapbiomas_df not found. Run the cell that loads the CSV first.')
# possible column names to consider for 'reference'
ref_candidates = ['reference', 'ref', 'Reference', 'REF']
ref_col = next((c for c in ref_candidates if c in mapbiomas_df.columns), None)
if ref_col is None:
    raise KeyError(f'No reference column found. Candidates tried: {ref_candidates}. Columns: {list(mapbiomas_df.columns)}')
df_ref15 = mapbiomas_df[mapbiomas_df[ref_col] == 15].copy()
print(f'Found {len(df_ref15)} rows with {ref_col} == 15')
# show first rows and the columns (for inspection)
df_ref15=df_ref15[df_ref15['year']>2010]
display(df_ref15.head())


Found 738208 rows with reference == 15


,reference,year,Lat,Lon
2214662,15,2011,-25.834198,-53.377393
2214663,15,2011,-25.880012,-51.709222
2214673,15,2011,-25.711847,-52.419879
2214676,15,2011,-24.547092,-51.203111
2214677,15,2011,-25.011700,-51.954462


In [7]:
df_sample = df_ref15.sample(n=300, random_state=42)
df_sample.head()
print(len(df_ref15))


182407


In [9]:
df_sample


,reference,year,Lat,Lon
2713545,15,2016,-12.073535,-43.897113
2641862,15,2016,-29.214738,-51.004224
2748140,15,2017,-19.577073,-52.182454
2889552,15,2018,-16.676772,-39.869516
2418105,15,2013,-10.618803,-55.163064
...,...,...,...,...
2539775,15,2014,-7.192718,-39.963031
2401929,15,2013,-12.199928,-60.820564
2704073,15,2016,-0.990570,-47.328318
2892354,15,2018,-19.147768,-44.306206


In [ ]:
# ...existing code...
import geopandas as gpd
from shapely.geometry import Point

df_sample = df_sample.reset_index(drop=True)
df_sample['ID'] = range(1, len(df_sample) + 1)

gdf = gpd.GeoDataFrame(
    df_sample,
    geometry=[Point(lon, lat) for lon, lat in zip(df_sample['Lon'], df_sample['Lat'])],
    crs="EPSG:4326",
)
gdf[['ID', 'year', 'geometry']].to_file('mapbiomas_reference_15_300.kml', driver='KML')
print('KML file saved as mapbiomas_reference_15_300.kml')
# ...existing code...


KML file saved as mapbiomas_reference_15_300.kml


In [ ]:
import os
print(os.getcwd())


/usr2/postdoc/chishan


In [ ]:
# Cell: export df_ref15 to a KML file usable by Google Earth
# This will try to detect latitude/longitude columns and write a simple KML with Placemarks
import html

def detect_latlon_columns(df):
    candidates_lat = ['Lat', 'latitude', 'LAT', 'Latitude', 'y', 'Y']
    candidates_lon = ['Lon', 'lng', 'longitude', 'LON', 'Longitude', 'x', 'X']
    lat_col = next((c for c in candidates_lat if c in df.columns), None)
    lon_col = next((c for c in candidates_lon if c in df.columns), None)
    return lat_col, lon_col

# require df_ref15 from previous cell
if 'df_ref15' not in globals():
    raise NameError('df_ref15 not found. Run the filtering cell first.')

lat_col, lon_col = detect_latlon_columns(df_sample)
if lat_col is None or lon_col is None:
    raise KeyError(f'Could not detect lat/lon columns. Tried lat candidates and lon candidates. Columns available: {list(df_sample.columns)}')

out_kml = 'mapbiomas_reference_15.kml'

def write_kml(df, lat_col, lon_col, out_path):
    with open(out_path, 'w', encoding='utf-8') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        f.write('<kml xmlns="http://www.opengis.net/kml/2.2">\n')
        f.write('<Document>\n')
        for idx, row in df.iterrows():
            name = f"Point {idx} - Year {row['year']}"
            lat = row[lat_col]
            lon = row[lon_col]
            f.write('<Placemark>\n')
            f.write(f'<name>{html.escape(name)}</name>\n')
            f.write('<Point>\n')
            f.write(f'<coordinates>{lon},{lat},0</coordinates>\n')
            f.write('</Point>\n')
            f.write('</Placemark>\n')
        f.write('</Document>\n')
        f.write('</kml>\n')

write_kml(df_sample, lat_col, lon_col, out_kml)
print(f'KML file saved as {out_kml}')


## Protected Area

In [ ]:
## Protected area
import ee 
import geemap
import os
ee.Initialize(project='ee-zcs')


In [ ]:
Map = geemap.Map()
Map.layout.height = '800px'
Map.add_basemap('HYBRID')
Map


In [ ]:
dataset = ee.FeatureCollection('WCMC/WDPA/current/polygons')
visParams = {
  'palette': ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
  'min': 0.0,
  'max': 1550000.0,
  'opacity': 0.8,
}
image = ee.Image().float().paint(dataset, 'REP_AREA')
Map.setCenter(41.104, -17.724, 6)
Map.addLayer(image, visParams, 'WCMC/WDPA/current/polygons')
Map.addLayer(dataset, {}, 'for Inspector', False)

# Define CODF_polygons and CODF_lines if not already defined
CODF_polygons = ee.FeatureCollection("projects/ee-zcs/assets/CODF_polygons")
CODF_lines = ee.FeatureCollection("projects/ee-zcs/assets/CODF_lines")

Map.addLayer(CODF_polygons, {}, 'polygons')

codf_polygons_buff = CODF_polygons.map(lambda feature: feature.buffer(25000.0))
codf_lines_buff = CODF_lines.map(lambda feature: feature.buffer(25000.0))

Map.addLayer(codf_polygons_buff, {}, 'polygons_buffer')
Map.addLayer(codf_lines_buff, {}, 'lines_buffer')


In [ ]:
CODF_lines


In [ ]:
# read the saved file
availability_threshold = 90  # Set your threshold here
output_file = f'/usr2/postdoc/chishan/project_data/CODF/tmf_suitable_sites_90percent.csv'

good_sites = pd.read_csv(output_file)

# Load all CODF datasets - points, lines, and polygons
HANSEN = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")
CODF_polygons = ee.FeatureCollection("projects/ee-zcs/assets/CODF_polygons")
CODF_points = ee.FeatureCollection("projects/ee-zcs/assets/CODF_points")
CODF_lines = ee.FeatureCollection("projects/ee-zcs/assets/CODF_lines")

#combine all CODF datasets into a single FeatureCollection
CODF_combined = CODF_polygons.merge(CODF_lines)
print("Combined CODF FeatureCollection:", CODF_combined.size().getInfo())

#filter good sites based on the availability threshold
filtered_CODF_polygons = CODF_combined.filter(ee.Filter.inList('BU ID', good_sites['BU_ID'].tolist()))
print("Filtered CODF FeatureCollection with good TMF coverage:", filtered_CODF_polygons.size().getInfo())


In [ ]:
filtered_CODF_polygons_buff = filtered_CODF_polygons.map(lambda feature: feature.buffer(25000.0))
Map.addLayer(filtered_CODF_polygons_buff, {}, 'filtered polygons buffer')
